In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import LayerIntegratedGradients
import pickle

In [9]:
class NaiveInception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(NaiveInception, self).__init__()

        self.conv1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.conv1x1_3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=3, padding=1)
        )
        self.conv1x1_5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=1),
            nn.Conv2d(8, 16, kernel_size=5, padding=2)
        )
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)

        self.fc = nn.Sequential(
            nn.Linear(38416, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)  # Number of output classes
        )

    def forward(self, x):
        x1 = self.conv1x1(x)
        x2 = self.conv1x1_3x3(x)
        x3 = self.conv1x1_5x5(x)
        x4 = self.pool(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [10]:
# Create and train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create an instance of the Inception-like 3-stack model
in_channels = 1  # Input channels (e.g., for RGB images)
num_classes = 10  # Number of classes in your classification task
model = NaiveInception(in_channels, num_classes).to(device)
print(model)

NaiveInception(
  (conv1x1): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
  (conv1x1_3x3): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (conv1x1_5x5): Sequential(
    (0): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  )
  (pool): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (fc): Sequential(
    (0): Linear(in_features=38416, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [11]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_subset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_subset = datasets.MNIST('.', train=False, download=True, transform=transform)

# Define the size of the random subsets
train_indices = torch.randperm(len(train_subset))[:30000]
test_indices = torch.randperm(len(test_subset))[:1000]

train_loader = torch.utils.data.DataLoader(train_subset, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=16, sampler=torch.utils.data.SubsetRandomSampler(test_indices))

# Create and train the model on the GPU
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


In [12]:
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")


def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

In [13]:
# Train for 5 epochs as an example
'''for epoch in range(5):
    train(epoch)
    test(epoch)
file_path = '/content/drive/My Drive/InterpretingNN/code/model saved/naiveinception1.pth'

# Save the model to a file
torch.save(model.state_dict(), file_path)'''

Epoch 0: Train Loss = 0.5237, Train Accuracy = 84.12%
Epoch 0: Test Loss = 0.3307, Test Accuracy = 91.20%
Epoch 1: Train Loss = 0.2516, Train Accuracy = 92.54%
Epoch 1: Test Loss = 0.2544, Test Accuracy = 92.50%
Epoch 2: Train Loss = 0.1829, Train Accuracy = 94.49%
Epoch 2: Test Loss = 0.1947, Test Accuracy = 93.90%
Epoch 3: Train Loss = 0.1430, Train Accuracy = 95.72%
Epoch 3: Test Loss = 0.1725, Test Accuracy = 94.40%
Epoch 4: Train Loss = 0.1154, Train Accuracy = 96.55%
Epoch 4: Test Loss = 0.1501, Test Accuracy = 95.20%


In [19]:
# Load the model's state dictionary from the file
model = NaiveInception(in_channels, num_classes).to(device)
file_path = '/content/drive/My Drive/InterpretingNN/code/model saved/naiveinception1.pth'
model.load_state_dict(torch.load(file_path))

# Put the model in evaluation mode (if needed)
model.eval()
print("loaded")

loaded


In [20]:
def calculate_integrated_gradients_one_layer(model, layer, input_data, target_class):
    lig = LayerIntegratedGradients(model, layer)
    attribution = lig.attribute(input_data, target=target_class)

    return attribution

In [21]:
def get_all_attributions_one_input(model, input_data, target_class):
    all_attributions = {}
    for name, layer in model.named_children():
      print(name)
      all_attributions[layer] = []
      layer_attributions=calculate_integrated_gradients_one_layer(model, layer, input_data, target_class)
      all_attributions[layer]=layer_attributions
    return all_attributions

In [22]:
sample_input, target_class = next(iter(test_loader))
print(target_class)
sample_input.requires_grad_()
device = torch.device("cpu")
sample_input=sample_input.to(device)
model.to(device)
mode=model
num_class=10
# Calculate and print importance scores for each layer and neuron
attributions=get_all_attributions_one_input(model, sample_input, target_class)
file_path = 'attributions.pkl'

# Save the dictionary to a Pickle file
with open(file_path, 'wb') as file:
    pickle.dump(attributions, file)

tensor([7, 6, 7, 9, 1, 2, 5, 3, 6, 6, 1, 2, 5, 8, 8, 2])
conv1x1
conv1x1_3x3
conv1x1_5x5
pool
fc


In [29]:
for i in attributions:
  print(i)
  print(attributions[i].sum().item())
  print(attributions[i].shape)
  print()

Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1))
34.60726481904782
torch.Size([16, 16, 28, 28])

Sequential(
  (0): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)
70.80325283791903
torch.Size([16, 16, 28, 28])

Sequential(
  (0): Conv2d(1, 8, kernel_size=(1, 1), stride=(1, 1))
  (1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)
81.9837445059679
torch.Size([16, 16, 28, 28])

MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
11.371645393883457
torch.Size([16, 1, 28, 28])

Sequential(
  (0): Linear(in_features=38416, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
170.94963979721072
torch.Size([16, 10])



In [ ]:
# Load the dictionary from the Pickle file
'''with open(file_path, 'rb') as file:
    loaded_dict = pickle.load(file)

print("Loaded Dictionary (Pickle):", loaded_dict)'''